<a href="https://colab.research.google.com/github/katlynkenisha/Spotify/blob/main/Spotify_Extracting_Playlist_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract prepared playlist using Spotify API

In [ ]:
!pip install spotipy

In [ ]:
# Connecting to spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = 'YOUR_client_id'
client_secret = 'YOUR_client_secret'

client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
# Function to call playlist
def call_playlist(username, playlist_id):
    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks

    playlist_tracks = []

    for i in range(len(results)):
        print(i)  # Counter
        if i == 0:
            artist_list = []
            genre_list = []
            for artist in results[i]['track']['artists']:
                artist_details = sp.artist(artist['id'])
                artist_list.append(artist['name'])
                genre_list.extend(artist_details['genres'])
            playlist_tracks_artists = ", ".join(artist_list)
            playlist_tracks_titles = results[i]['track']['name']
            playlist_tracks_id = results[i]['track']['id']
            playlist_tracks_release_date = results[i]['track']['album']['release_date']
            playlist_tracks_popularity = results[i]['track']['popularity']

            features = sp.audio_features(playlist_tracks_id)
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['artist'] = playlist_tracks_artists
            features_df['track_title'] = playlist_tracks_titles
            features_df['track_id'] = playlist_tracks_id
            features_df['release_date'] = playlist_tracks_release_date
            features_df['popularity'] = playlist_tracks_popularity
            features_df['genre'] = ", ".join(genre_list)
            features_df = features_df[['artist', 'track_title', 'track_id', 'release_date', 'popularity', 'genre',
                                       'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness',
                                       'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'valence']]
            continue
        else:
            try:
                artist_list = []
                genre_list = []
                for artist in results[i]['track']['artists']:
                    artist_details = sp.artist(artist['id'])
                    artist_list.append(artist['name'])
                    genre_list.extend(artist_details['genres'])
                playlist_tracks_artists = ", ".join(artist_list)
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_release_date = results[i]['track']['album']['release_date']
                playlist_tracks_popularity = results[i]['track']['popularity']
                features = sp.audio_features(playlist_tracks_id)
                new_row = {'artist': [playlist_tracks_artists],
                           'track_title': [playlist_tracks_titles],
                           'track_id': [playlist_tracks_id],
                           'release_date': [playlist_tracks_release_date],
                           'popularity': [playlist_tracks_popularity],
                           'genre': ", ".join(genre_list),
                           'acousticness': [features[0]['acousticness']],
                           'danceability': [features[0]['danceability']],
                           'duration_ms': [features[0]['duration_ms']],
                           'energy': [features[0]['energy']],
                           'instrumentalness': [features[0]['instrumentalness']],
                           'key': [features[0]['key']],
                           'liveness': [features[0]['liveness']],
                           'loudness': [features[0]['loudness']],
                           'mode': [features[0]['mode']],
                           'speechiness': [features[0]['speechiness']],
                           'tempo': [features[0]['tempo']],
                           'valence': [features[0]['valence']]
                           }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index=True)
            except:
                continue

    return features_df

In [ ]:
# Call playlist
df = call_playlist('OWNER','PLAYLIST ID')

In [ ]:
# Dataframe preview
df.head(2)

# Download selected tracks as dataframe to excel

In [ ]:
# Download dataframe to excel
from google.colab import files

df.to_excel('spotify.xlsx', encoding = 'utf-8-sig')
files.download('spotify.xlsx')